In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import time 
from sklearn import manifold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import svm
%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

T-SNE

In [ ]:
df_train = pd.read_csv("fashion-mnist_train.csv")
df_test = pd.read_csv("fashion-mnist_test.csv")
df_train_pixels = df_train.iloc[:,1:] 
df_target = df_train.iloc[:,0]

In [ ]:
#Using first 10000 data to do T-SNE.
tsne = manifold.TSNE(n_components=2, random_state=1)
transf_data = tsne.fit_transform(df_train_pixels[:10000].to_numpy())
df_tsne = pd.DataFrame(np.column_stack((transf_data, df_target[:10000])),
                      columns = ["X", "y", "label"])
df_tsne.head()

In [ ]:
fig= plt.figure(figsize=(12,10))
sns.scatterplot(data=df_tsne, x="X", y="y", hue="label", palette="deep")

In [ ]:
#Using all data to do T-SNE.
tsne = manifold.TSNE(n_components=2, random_state=1)
transf_dataall = tsne.fit_transform(df_train_pixels.to_numpy())
df_tsneall = pd.DataFrame(np.column_stack((transf_dataall, df_target)),
                      columns = ["X", "y", "label"])
df_tsneall.head()

In [ ]:
fig= plt.figure(figsize=(12,10))
sns.scatterplot(data=df_tsneall, x="X", y="y", hue="label", palette="deep")

LightGBM

In [ ]:
# Perform train-test split
X_train, y_train, X_test, y_test = df_train.iloc[:,1:], df_train['label'], df_test.iloc[:,1:], df_test['label']
X_train.head()

In [ ]:
training_times = []
training_acc = []
testing_acc = []

In [ ]:
def Train(clf, X, y, X_predict, y_predict):
    # Train
    start = time.time()
    model = clf.fit(X,y)
    end = time.time()
    print('Training time: ',end-start)
    training_times.append(round((end - start),2))
    # Predict
    train_acc = accuracy_score(y, model.predict(X))
    training_acc.append(round(train_acc,4))
    print('Training Accuracy {}'.format(train_acc))
    test_acc = accuracy_score(y_predict, model.predict(X_predict))
    testing_acc.append(round(test_acc,4))
    print('Testing Accuracy {}'.format(test_acc))

In [ ]:
for n in [5,10,25,50,100,150,200,250,300,400,500,600,800,1000,1250,1500,2000]:
    print('Using {} estimators.'.format(n))
    Train(LGBMClassifier(n_estimators=n), X_train, y_train, X_test, y_test)

In [ ]:
#The best parameter is 1500 estimators.
clf = LGBMClassifier(n_estimators=1500)
model = clf.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
confusion = confusion_matrix(y_test,y_test_pred)

In [ ]:
#Plot Confusion Matrix
fig = plt.figure(figsize=(8, 5))
conf_mat = np.matrix(confusion/np.sum(confusion, axis=1))
ax = sns.heatmap(conf_mat, annot=True, fmt ='.2%',cmap='Blues')
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted label')
ax.set_ylabel('True label')
plt.show()

Ensemble

In [ ]:
#Train Test Set Split
X_train, y_train, X_test, y_test = df_train.iloc[:,1:], df_train['label'], df_test.iloc[:,1:], df_test['label']

In [ ]:
#Normalize 
scaler = StandardScaler()
X_train_sd = scaler.fit_transform(X_train)
X_test_sd = scaler.fit_transform(X_test)

In [ ]:
#Train SVM
s_m = svm.SVC(gamma=0.001, C = 10, kernel='rbf')
s_m.fit(X_train_sd, y_train)
s_m_test_pred = s_m.predict(X_test_sd)
accuracy_score(y_test,s_m_test_pred)

In [ ]:
#Train LightGBM
lgb = LGBMClassifier(n_estimators=1500)
lgb.fit(X_train, y_train)
lgb_test_pred = lgb.predict(X_test)
accuracy_score(y_test,lgb_test_pred)

In [ ]:
#Train XGBoost
xgb = XGBClassifier(n_estimators=1000)
xgb.fit(X_train_sd, y_train)
xgb_test_pred = xgb.predict(X_test_sd)
accuracy_score(y_test,xgb_test_pred)

In [ ]:
def func(x):
    if len(np.unique(x.values))<=2:
        if x[0]==x[1]:
            return x[0]
        elif x[0]==x[2]:
            return x[0]
        else:
            return x[2]
    else:
        return x[2]

In [ ]:
#Max Voting ensemble
test_pred_ensemble = np.hstack([xgb_test_pred.reshape(-1,1),s_m_test_pred.reshape(-1,1),lgb_test_pred.reshape(-1,1)])
test_pred_ensemble_df = pd.DataFrame(test_pred_ensemble)
test_pred_ensemble_df_predict = test_pred_ensemble_df.apply(lambda x:func(x),axis=1)
accuracy_score(y_test,test_pred_ensemble_df_predict)

In [ ]:
#Plot Confusion Matrix
confusion = confusion_matrix(y_test,test_pred_ensemble_df_predict)
fig = plt.figure(figsize=(8, 5))
conf_mat = np.matrix(confusion/np.sum(confusion, axis=1))
ax = sns.heatmap(conf_mat2, annot=True, fmt ='.2%',cmap='Blues')
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted label')
ax.set_ylabel('True label')
plt.show()